In [228]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix # "classification_report" import 불가 
import streamlit as st


In [229]:
# 폰트지정
plt.rcParams["font.family"] = "NanumGothic"

# 마이너스 부호 깨짐 지정
plt.rcParams["axes.unicode_minus"] = False

# 숫자가 지수표현식으로 나올 때 지정
pd.options.display.float_format = "{:.2f}".format

In [230]:
# 데이터 로드
file_path1 = "../dataset_miniProject/miniProject_coldcode_city.csv" # 데이터 파일 경로
file_path2 = "../dataset_miniProject/humidity_data.csv"
file_path3 = "../dataset_miniProject/rain_data.csv"
file_path4 = "../dataset_miniProject/temperature_data.csv"
df1 = pd.read_csv(file_path1, encoding="cp949")
df2 = pd.read_csv(file_path2, encoding="cp949")
df3 = pd.read_csv(file_path3, encoding="cp949")
df4 = pd.read_csv(file_path4, encoding="cp949")

In [231]:
# 기본 데이터 정보 확인
print("데이터 크기:", df1.shape)
print("데이터 크기:", df2.shape)
print("데이터 크기:", df3.shape)
print("데이터 크기:", df4.shape)
print("\n처음 5개 행(진료데이터):")
display(df1.head())
print("\n처음 5개 행(습도):")
display(df2.head())
print("\n처음 5개 행(강수):")
display(df3.head())
print("\n처음 5개 행(기온):")
display(df4.head())
print("\n데이터 기간:", df1["날짜"].min(), "~", df1["날짜"].max())
print("총 관측 수:", len(df1))
print("\n데이터 기간:", df2["날짜"].min(), "~", df2["날짜"].max())
print("총 관측 수:", len(df2))
print("\n데이터 기간:", df3["날짜"].min(), "~", df3["날짜"].max())
print("총 관측 수:", len(df3))
print("\n데이터 기간:", df4["날짜"].min(), "~", df4["날짜"].max())
print("총 관측 수:", len(df4))

데이터 크기: (63078, 3)
데이터 크기: (120, 3)
데이터 크기: (120, 6)
데이터 크기: (120, 9)

처음 5개 행(진료데이터):


,날짜,시도지역코드,발생건수(건)
0,2014.1.1,11,5992
1,2014.1.1,26,3158
2,2014.1.1,27,2190
3,2014.1.1,28,3426
4,2014.1.1,29,2635



처음 5개 행(습도):


,날짜,평균습도(%rh),최저습도(%rh)
0,01-15,56,22
1,02-15,59,17
2,03-15,45,9
3,04-15,55,11
4,05-15,53,12



처음 5개 행(강수):


,날짜,강수량(mm),일최다강수량(mm),일최다강수량일자,1시간최다강수량(mm),1시간최다강수량일자
0,01-15,11.30,4.00,2015-01-18,NaN,NaN
1,02-15,22.70,10.50,2015-02-16,NaN,NaN
2,03-15,9.60,5.50,2015-03-03,NaN,NaN
3,04-15,80.50,28.00,2015-04-02,16.50,2015-04-02
4,05-15,28.90,14.00,2015-05-11,6.50,2015-05-11



처음 5개 행(기온):


,날짜,평균기온(℃),평균최고기온(℃),최고기온(℃),최고기온일자,평균최저기온(℃),최저기온(℃),최저기온일자,평균 일교차
0,01-15,-0.90,3.60,9.00,2015.1.15,-4.80,-9.80,2015.1.1,8.40
1,02-15,1.00,6.00,13.00,2015.2.15,-2.90,-13.00,2015.2.9,8.90
2,03-15,6.30,12.40,21.90,2015.3.20,1.00,-6.90,2015.3.10,11.40
3,04-15,13.30,18.80,28.30,2015.4.30,8.40,3.50,2015.4.8,10.40
4,05-15,18.90,24.90,32.20,2015.5.28,13.60,8.30,2015.5.5,11.30



데이터 기간: 2014.1.1 ~ 2024-02-29
총 관측 수: 63078

데이터 기간: 01-15 ~ 12-24
총 관측 수: 120

데이터 기간: 01-15 ~ 12-24
총 관측 수: 120

데이터 기간: 01-15 ~ 12-24
총 관측 수: 120


In [232]:
# 데이터 전처리

# 진료건수
# 날짜 형식 변경
# 날짜를 0000-00-00 형태의 datetime으로 변경
# 일 단위 삭제(문자열로 변경) 후, 다시 datetime으로 변경(일이 01일로 통일)
df1["날짜"] = df1["날짜"].str.replace(r"[.-]", "-", regex=True)
df1["날짜"] = pd.to_datetime(df1["날짜"], format="%Y-%m-%d")
df1["날짜"] = df1["날짜"].dt.strftime('%Y-%m')
df1["날짜"] = pd.to_datetime(df1["날짜"], format="%Y-%m")
# print(df1["날짜"])
# print(df1["날짜"].dtype)

# 날짜 기준 설정 및 적용
min_date1 = pd.to_datetime("2014-12")
max_date1 = pd.to_datetime("2023-12")
coldConsultation_filteredByDate = df1[(df1["날짜"] > min_date1) & (df1["날짜"] <= max_date1)]

# 결측치 처리
# 결측치를 평균 값으로 채우겠다
coldConsultation_filteredByDate["발생건수(건)"] = coldConsultation_filteredByDate["발생건수(건)"].fillna(coldConsultation_filteredByDate["발생건수(건)"].mean())

# 지역코드 11(서울)만 추출
coldConsultation_seoul = coldConsultation_filteredByDate[coldConsultation_filteredByDate["시도지역코드"] == 11]
print("\n지역코드 추출 데이터 set:")
print(coldConsultation_seoul)

# 동일 날짜의 모든지역 진료건수 더하기(월별 데이터로 변환)
coldConsultation_seoul = coldConsultation_seoul.groupby("날짜")["발생건수(건)"].sum().reset_index()
print("\n동일 날짜 기준 데이터 set:")
print(coldConsultation_seoul)

# 날씨
# 날짜 형식 변경 함수
# 연도별로 달리 처리: 2015년은 15로, 그 외는 연도의 끝 2자리를 사용
def convert_date_format(date_str):
    month, year_end = date_str.split("-")
    
    # 2015년인 경우
    if year_end == "15":
        return f"2015-{str(month).zfill(2)}"
    
    # 2016년 이후는 연도 끝 두 자리를 이용해 "2016-01", "2017-02" 형태로 변환
    return f"20{year_end}-{str(month).zfill(2)}"

# 날짜 형식 변경
df2["날짜"] = df2["날짜"].apply(convert_date_format)
df3["날짜"] = df3["날짜"].apply(convert_date_format)
df4["날짜"] = df4["날짜"].apply(convert_date_format)
df2["날짜"] = pd.to_datetime(df2["날짜"], format="%Y-%m")
df3["날짜"] = pd.to_datetime(df3["날짜"], format="%Y-%m")
df4["날짜"] = pd.to_datetime(df4["날짜"], format="%Y-%m")

# 날짜 기준 설정 및 적용
df2["날짜"] = pd.to_datetime(df2["날짜"], format="%Y-%m")
df3["날짜"] = pd.to_datetime(df3["날짜"], format="%Y-%m")
df4["날짜"] = pd.to_datetime(df4["날짜"], format="%Y-%m")
min_date = pd.to_datetime("2014-12")
max_date = pd.to_datetime("2023-12")
seoulHumidity_filteredByDate = df2[(df2["날짜"] > min_date) & (df2["날짜"] <= max_date)]
seoulRain_filteredByDate = df3[(df3["날짜"] > min_date) & (df3["날짜"] <= max_date)]
seoulTemperature_filteredByDate = df4[(df4["날짜"] > min_date) & (df4["날짜"] <= max_date)]
print("\n습도 데이터 set:")
print(seoulHumidity_filteredByDate)
print("\n강수 데이터 set:")
print(seoulRain_filteredByDate)
print("\n기온 데이터 set:")
print(seoulTemperature_filteredByDate)

# 날짜를 기준으로 두 데이터값 merge
data_merge_humidity = pd.merge(coldConsultation_seoul, seoulHumidity_filteredByDate, on="날짜", how="inner")
data_merge_rain = pd.merge(data_merge_humidity, seoulRain_filteredByDate, on="날짜", how="inner")
data_merge_total = pd.merge(data_merge_rain, seoulTemperature_filteredByDate, on="날짜", how="inner")
print("\n최종 데이터 set:")
print(data_merge_total)

# NaN값을 평균값으로 대체
data_merge_total["1시간최다강수량(mm)"] = data_merge_total["1시간최다강수량(mm)"].fillna(data_merge_total["1시간최다강수량(mm)"].mean())

# 날짜를 숫자형으로 변환
data_merge_total['날짜(int)'] = data_merge_total['날짜'].astype(int) / 10**9  # Unix 타임스탬프로 변환 (초 단위)


지역코드 추출 데이터 set:
              날짜  시도지역코드  발생건수(건)
6205  2015-01-01      11     6114
6222  2015-01-01      11   113444
6239  2015-01-01      11    70757
6256  2015-01-01      11     8263
6273  2015-01-01      11   108801
...          ...     ...      ...
61981 2023-12-01      11   119619
61998 2023-12-01      11   117050
62015 2023-12-01      11   143138
62032 2023-12-01      11    95235
62049 2023-12-01      11    16561

[3287 rows x 3 columns]

동일 날짜 기준 데이터 set:
            날짜  발생건수(건)
0   2015-01-01  2104489
1   2015-02-01  2134695
2   2015-03-01  2421135
3   2015-04-01  2363373
4   2015-05-01  1906589
..         ...      ...
103 2023-08-01  1708138
104 2023-09-01  1881361
105 2023-10-01  2592839
106 2023-11-01  3040420
107 2023-12-01  3307178

[108 rows x 2 columns]

습도 데이터 set:
            날짜  평균습도(%rh)  최저습도(%rh)
0   2015-01-01         56         22
1   2015-02-01         59         17
2   2015-03-01         45          9
3   2015-04-01         55         11
4   2015-05-01      

/var/folders/t5/w4syjrms3gn00d04kqv8dbyr0000gn/T/ipykernel_73908/1571582089.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  coldConsultation_filteredByDate["발생건수(건)"] = coldConsultation_filteredByDate["발생건수(건)"].fillna(coldConsultation_filteredByDate["발생건수(건)"].mean())


In [233]:
# 전처리된 데이터 확인
print("\n전처리된 데이터 샘플:")
display(data_merge_total.head())


전처리된 데이터 샘플:


,날짜,발생건수(건),평균습도(%rh),최저습도(%rh),강수량(mm),일최다강수량(mm),일최다강수량일자,1시간최다강수량(mm),1시간최다강수량일자,평균기온(℃),평균최고기온(℃),최고기온(℃),최고기온일자,평균최저기온(℃),최저기온(℃),최저기온일자,평균 일교차,날짜(int)
0,2015-01-01,2104489,56,22,11.30,4.00,2015-01-18,20.50,NaN,-0.90,3.60,9.00,2015.1.15,-4.80,-9.80,2015.1.1,8.40,1420070400.00
1,2015-02-01,2134695,59,17,22.70,10.50,2015-02-16,20.50,NaN,1.00,6.00,13.00,2015.2.15,-2.90,-13.00,2015.2.9,8.90,1422748800.00
2,2015-03-01,2421135,45,9,9.60,5.50,2015-03-03,20.50,NaN,6.30,12.40,21.90,2015.3.20,1.00,-6.90,2015.3.10,11.40,1425168000.00
3,2015-04-01,2363373,55,11,80.50,28.00,2015-04-02,16.50,2015-04-02,13.30,18.80,28.30,2015.4.30,8.40,3.50,2015.4.8,10.40,1427846400.00
4,2015-05-01,1906589,53,12,28.90,14.00,2015-05-11,6.50,2015-05-11,18.90,24.90,32.20,2015.5.28,13.60,8.30,2015.5.5,11.30,1430438400.00


In [234]:
data_merge_total["year"] = data_merge_total["날짜"].dt.year
data_merge_total["month"] = data_merge_total["날짜"].dt.month
display(data_merge_total.head())

,날짜,발생건수(건),평균습도(%rh),최저습도(%rh),강수량(mm),일최다강수량(mm),일최다강수량일자,1시간최다강수량(mm),1시간최다강수량일자,평균기온(℃),평균최고기온(℃),최고기온(℃),최고기온일자,평균최저기온(℃),최저기온(℃),최저기온일자,평균 일교차,날짜(int),year,month
0,2015-01-01,2104489,56,22,11.30,4.00,2015-01-18,20.50,NaN,-0.90,3.60,9.00,2015.1.15,-4.80,-9.80,2015.1.1,8.40,1420070400.00,2015,1
1,2015-02-01,2134695,59,17,22.70,10.50,2015-02-16,20.50,NaN,1.00,6.00,13.00,2015.2.15,-2.90,-13.00,2015.2.9,8.90,1422748800.00,2015,2
2,2015-03-01,2421135,45,9,9.60,5.50,2015-03-03,20.50,NaN,6.30,12.40,21.90,2015.3.20,1.00,-6.90,2015.3.10,11.40,1425168000.00,2015,3
3,2015-04-01,2363373,55,11,80.50,28.00,2015-04-02,16.50,2015-04-02,13.30,18.80,28.30,2015.4.30,8.40,3.50,2015.4.8,10.40,1427846400.00,2015,4
4,2015-05-01,1906589,53,12,28.90,14.00,2015-05-11,6.50,2015-05-11,18.90,24.90,32.20,2015.5.28,13.60,8.30,2015.5.5,11.30,1430438400.00,2015,5


In [235]:
# 감기 진료 건수 모델 학습
features_coldConsultation = ["평균습도(%rh)", "강수량(mm)", "평균기온(℃)", "평균 일교차"]
X_coldConsultation = data_merge_total[features_coldConsultation]
y_coldConsultation = data_merge_total["발생건수(건)"]
print(X_coldConsultation)
print(y_coldConsultation)

     평균습도(%rh)  강수량(mm)  평균기온(℃)  평균 일교차
0           56    11.30    -0.90    8.40
1           59    22.70     1.00    8.90
2           45     9.60     6.30   11.40
3           55    80.50    13.30   10.40
4           53    28.90    18.90   11.30
..         ...      ...      ...     ...
103         76   298.10    27.20    6.50
104         74   134.50    23.70    7.40
105         67    31.00    15.80    9.50
106         65    81.90     6.80    8.60
107         69    85.90     1.10    7.70

[108 rows x 4 columns]
0      2104489
1      2134695
2      2421135
3      2363373
4      1906589
        ...   
103    1708138
104    1881361
105    2592839
106    3040420
107    3307178
Name: 발생건수(건), Length: 108, dtype: int64


In [242]:
# 데이터 분할
X_train_coldConsultation, X_test_coldConsultation, y_train_coldConsultation, y_test_coldConsultation = train_test_split(X_coldConsultation, y_coldConsultation, test_size=0.2, random_state=42)
# 데이터를 표준화합니다.
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_coldConsultation)
X_test_scaled = scaler.transform(X_test_coldConsultation)
print(X_train_scaled)

[[ 7.87175330e-01  1.52878848e-01  4.66617095e-01  3.00499254e-01]
 [-1.49340001e+00 -8.62509675e-01 -7.26547126e-01  1.06058560e+00]
 [-4.13127478e-01 -7.77003273e-01 -6.75557202e-01 -3.07569825e-01]
 [ 1.26729645e+00  1.94656996e+00  1.27225789e+00 -1.82774252e+00]
 [-1.73346057e+00 -7.79918264e-01 -1.43020808e+00 -3.07569825e-01]
 [ 3.07054207e-01  5.72637549e-01  6.19586867e-01  9.08568332e-01]
 [-6.53188040e-01 -1.93033414e-01  6.88956880e-02  1.51663741e+00]
 [ 6.69936451e-02 -2.73681498e-01  8.23546563e-01 -6.11604364e-01]
 [-6.53188040e-01 -8.66396330e-01  5.78794928e-01  1.44062878e+00]
 [ 1.50735701e+00  2.88714038e+00  1.00711029e+00 -1.14366481e+00]
 [ 6.69936451e-02 -1.47365222e-01  2.11667475e-01  9.08568332e-01]
 [-5.33157759e-01 -6.81780234e-01  3.03449338e-01  1.28861151e+00]
 [ 1.02723589e+00  9.11748166e-01  1.40483170e+00 -9.15638903e-01]
 [ 1.86744786e+00  1.95725826e+00  1.40483170e+00 -2.05576843e+00]
 [-1.01327888e+00 -7.82833255e-01  6.19586867e-01  1.36462014e

In [237]:
# 모델 학습
model_coldConsultation = LinearRegression()
model_coldConsultation.fit(X_train_scaled, y_train_coldConsultation)

LinearRegression()

In [238]:
model_coldConsultation.score(X_test_scaled, y_test_coldConsultation)
print(model_coldConsultation.score(X_test_scaled, y_test_coldConsultation))

0.26437030705228537


In [239]:
# 모델 예측
y_pred_coldConsultation = model_coldConsultation.predict(X_test_scaled)
print(y_pred_coldConsultation)

[1432767.40384419 1555372.79486718 2114097.9007074  2000238.27724311
 2473534.52734804 1566757.72213988 1785052.57641947 2085486.88741864
 2145078.64615545 2059198.21718774 1586216.06268682 1428258.42350831
 2211829.77888991 1513188.49312755 1619767.54027138 1853382.82264731
 1714440.57417196 1784082.15219971 1899530.10912372 1589167.19803544
 2314875.51358423 2480071.02849085]


In [240]:
# 모델 평가
print("\ncoldConsultation 모델 성능")
print(f"R2 Score: {r2_score(y_test_coldConsultation, y_pred_coldConsultation):.4f}")
print(f"MSE: {mean_squared_error(y_test_coldConsultation, y_pred_coldConsultation):.4f}")


coldConsultation 모델 성능
R2 Score: 0.2644
MSE: 410046893740.1334


In [241]:
humidity = st.number_input("습도(%) : ", min_value=0, max_value=100, step=5)
rain = st.number_input("강수량(mm) : ", min_value=0, max_value=500, step=10)
temperature = st.number_input("기온(℃) : ", min_value=-30, max_value=50, step=1)
temperature_range = st.number_input("일교차(℃) : ", min_value=0, max_value=40, step=1)

future_date = pd.DataFrame({
    "평균습도(%rh)": [humidity],        # 예: 2025년 3월 예상 평균 습도
    "강수량(mm)": [rain],        # 예: 2025년 3월 예상 강수량
    "평균기온(℃)": [temperature],       # 예: 2025년 3월 예상 평균 기온
    "평균 일교차": [temperature_range],        # 예: 2025년 3월 예상 평균 일교차
})

# 모델 예측
if st.button("예측 감기 발병 건수"):
    future_date_scaled = scaler.transform(future_date)
    predicted_coldConsultation = model_coldConsultation.predict(future_date_scaled)
    st.write(f"날씨에 따른 감기 발병 건수(예측): {round(predicted_coldConsultation[0], 2)}")

2025-01-07 14:49:01.320 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-07 14:49:01.321 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-07 14:49:01.321 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-07 14:49:01.322 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-07 14:49:01.322 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`
2025-01-07 14:49:01.322 WARNING streamlit.runtime.scriptrunner_utils.script_run_c

날씨에 따른 감기 발병 건수(예측): 4170121.775323473
